# Tutorial for MFedNI(FGCS'26)
🧑‍🎓 **Author**: [Kaiming Zhu](https://kaimingzhu.github.io/)

📅 **Date**: 29/11/2025

🌟 **Mention**: Please generate the HAR dataset before you run this script. You could refer to `./.dataset` for how to generate it.

Hi, this is a simple tutorial about how we implement MFedNI with the **HAR dataset**. 

Here are details of the **HAR dataset** we evaluated in this showcase.

- **Data distribution**: Non-IID, Dirichlet Distributed ($\alpha=2.0$).
- **The** *ratio* **of data has complete modal**: 14%.
- **Amount of FL clients**: 20.

### 1. Import packages

In [ ]:
import warnings
warnings.filterwarnings("ignore")


# packages importing
import os
from typing import Tuple, Optional, List

import numpy as np
import scipy.io as sio
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from fldataset.converter.torch import MultiModalDataset
from model.torch import ModelFactory, Initializer
from mfedni import Simulator, Client

### 2. Hyper-params specifications

In [2]:
# dataset settings
# hyper-param: where to load dataset
dataset_path = "./.dataset/HAR/splited/"
# hyper-param: keys of each modal data
data_keys = ["acc_feats", "gyro_feats"]
client_count: int = 20
transform = None
target_transform = None

# training config
model = ModelFactory.CMTFFModel
initializer = Initializer.KaimingNormal
lr = 0.05
batch_size = 32
optimizer = optim.SGD
criterion = nn.CrossEntropyLoss
momentum: Optional[float] = 0.9
weight_decay: Optional[float] = 1e-5
need_dataset_shuffle = False
alpha: Optional[float] = 0.01 

# global training config
global_epoch = 100
local_epoch = 10
device = "cuda:0" if torch.cuda.is_available() else "cpu"
need_checkpoint_archiving = False
each_round_client_amount = None
need_client_shuffle = False

# utils for pytorch training acceleration
num_workers = 0
pin_memory = False

### 3. Load dataset
These code will load pre-generated datasets, and generate two different `List` object.
- `trainsets`: the train set of each client (w.r.t. their indices)
- `trainsets`: the test set of each client (w.r.t. their indices)

In [3]:
# uniform w.r.t. dictionary-order
data_keys = sorted(data_keys)

# Load dataset
filenames = list(os.listdir(dataset_path))
filenames = [filename for filename in filenames if filename.endswith(".mat")]
def truncate_index_from_name(filename):
    idx_start = filename.index("client") + len("client")
    idx_end = filename.index(".mat")
    return int(filename[idx_start: idx_end])
filenames = sorted(filenames, key=truncate_index_from_name)[0: client_count]

# archive as a nested list
each_client_train_datas: List[List[np.ndarray]] = []
each_client_train_labels: List[np.ndarray] = []
each_client_test_datas: List[List[np.ndarray]] = []
each_client_test_labels: List[np.ndarray] = []
for filename in filenames:
    data_by_key = sio.loadmat(dataset_path + filename, squeeze_me=True)
    train_datas = [data_by_key[key] for key in data_keys]
    train_labels = data_by_key["labels"]
    each_client_train_datas.append(train_datas)
    each_client_train_labels.append(train_labels)

    testset_keys = ["test_" + key for key in data_keys]
    test_datas = [data_by_key[key] for key in testset_keys]
    test_labels = data_by_key["test_labels"]
    each_client_test_datas.append(test_datas)
    each_client_test_labels.append(test_labels)

# convert to torch.Dataset
trainsets: List[MultiModalDataset] = []
for train_datas, train_labels in zip(each_client_train_datas, each_client_train_labels):
    trainset = MultiModalDataset(
        modal_data_by_key={key: data for key, data in zip(data_keys, train_datas)},
        targets=train_labels
    )
    trainsets.append(trainset)

testsets: List[MultiModalDataset] = []
for test_datas, test_labels in zip(each_client_test_datas, each_client_test_labels):
    testset = MultiModalDataset(
        modal_data_by_key={key: data for key, data in zip(data_keys, test_datas)},
        targets=test_labels
    )
    testsets.append(testset)

### 4. Generate Clients in Federated Learning (FL)
These codes will generate clients in FL, they will be stored in `clients`.

In [4]:
# get input / output shape of model
output_shape: Tuple[int,] = (np.max(np.concatenate(each_client_train_labels)) + 1,)
input_shapes: List[Tuple] = []
for modal_data in each_client_train_datas[0]:
    input_shapes.append(modal_data.shape[1:])

# make optimizer
def make_optimizer(target_model: nn.Module) -> optim.Optimizer:
    if optimizer is optim.SGD:
        return optimizer(params=target_model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    else:
        return optimizer(params=target_model.parameters(), lr=lr)

# generate clients
clients: List[Client] = []
for index, (trainset, testset) in enumerate(zip(trainsets, testsets)):
    train_loader = DataLoader(
        trainset,
        batch_size=min(batch_size, len(trainset)),
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=need_dataset_shuffle
    )
    test_loader = DataLoader(
        testset,
        batch_size=min(batch_size, len(testset)),
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=need_dataset_shuffle
    )

    client_model = model.multi_modal_instance(*input_shapes, output_shape=output_shape, initializer=None)
    client_model = client_model.to(device)

    client = Client(
        model=client_model,
        train_loader=train_loader,
        test_loader=test_loader,
        optimizer=make_optimizer(target_model=client_model),
        criterion=criterion().to(device),
        local_epoch=local_epoch,
        device=device,
        alpha=alpha
    )
    clients.append(client)

### 5. Generate FL simulator
These code will generate FL simulator, and register all clients into it.

In [5]:
global_model = model.multi_modal_instance(*input_shapes, output_shape=output_shape, initializer=initializer)
global_model = global_model.to(device)
simulator = Simulator(
    clients=clients,
    global_model=global_model,
    global_epochs=global_epoch,
    archive_folder_name=None,
    device=device,
    each_round_client_amount=each_round_client_amount,
    need_client_shuffle=need_client_shuffle
)
simulator.need_checkpoint_archiving = False

### 6. Run MFedNI
It will output the metrics in average (evaluated by all clients train-set and test-set).

In [6]:
simulator.run()

global epoch 0/100: train acc 12.32%, train loss 0.0601, test acc 14.23%, test loss 0.0808.
global epoch 1/100 [48.54s]: train acc 19.86%, train loss 0.0578; test acc 17.48%, test loss 0.0783.
global epoch 2/100 [57.63s]: train acc 37.31%, train loss 0.0380; test acc 32.93%, test loss 0.0546.
global epoch 3/100 [58.62s]: train acc 37.96%, train loss 0.0371; test acc 33.47%, test loss 0.0533.
global epoch 4/100 [58.36s]: train acc 32.34%, train loss 0.0374; test acc 31.98%, test loss 0.0526.
global epoch 5/100 [57.71s]: train acc 43.80%, train loss 0.0357; test acc 38.21%, test loss 0.0518.
global epoch 6/100 [58.91s]: train acc 50.53%, train loss 0.0368; test acc 45.80%, test loss 0.0504.
global epoch 7/100 [58.77s]: train acc 47.28%, train loss 0.0310; test acc 47.29%, test loss 0.0456.
global epoch 8/100 [58.58s]: train acc 54.12%, train loss 0.0262; test acc 51.90%, test loss 0.0408.
global epoch 9/100 [58.65s]: train acc 51.53%, train loss 0.0263; test acc 51.90%, test loss 0.0409.